In [43]:
import numpy as np
import scipy.stats as stats
import os
import pickle
import scipy.io as scio
from ncdump_python3 import ncdump
import pickle
import Area_Avg as AA
import matplotlib as mpl
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap
from scipy.interpolate import griddata
from netCDF4 import Dataset
from ncdump_python3 import ncdump
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

In [45]:
#Load in Dimensions
LLL = Dataset('LatLon.nc')
Lat = np.squeeze(LLL.variables['lat'])
Lon = np.squeeze(LLL.variables['lon'])
LLL.close()

In [3]:
Source_File = Dataset('Kernels/Interpolated/CAM3_Kernels.nc')

#CAM3 Horizontal resolution grid
CAM3_Lat = np.squeeze(Source_File.variables['lat'])
CAM3_Lon = np.squeeze(Source_File.variables['lon'])

In [4]:
wx,wy = np.meshgrid(WACCM4_Lon,WACCM4_Lat)
cx,cy = np.meshgrid(CAM3_Lon,CAM3_Lat)

In [5]:
RCP45_forc_FLNT_2060 = np.zeros([12,96,144])
RCP45_forc_FSNT_2060 = np.zeros([12,96,144])
RCP45_forc_FLNR_2060 = np.zeros([12,96,144])
RCP45_forc_FSNR_2060 = np.zeros([12,96,144])

RCP45_forc_FLNT_2006 = np.zeros([12,96,144])
RCP45_forc_FSNT_2006 = np.zeros([12,96,144])
RCP45_forc_FLNR_2006 = np.zeros([12,96,144])
RCP45_forc_FSNR_2006 = np.zeros([12,96,144])
Months = ['01','02','03','04','05','06','07','08','09','10','11','12']
for i in range(12):
    RCP45_forc_FLNT_2060[i,:,:] = np.squeeze(\
    Dataset('Port/y2060_RCP45/b2060.wcm.port.rcp45.pert.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FLNT'])
    
    RCP45_forc_FSNT_2060[i,:,:] = np.squeeze(\
    Dataset('Port/y2060_RCP45/b2060.wcm.port.rcp45.pert.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FSNT'])
    
    RCP45_forc_FLNT_2006[i,:,:] = np.squeeze(\
    Dataset('Port/y2006_RCP45/b2006.wcm.port.rcp45.baseline.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FLNT'])
    
    RCP45_forc_FSNT_2006[i,:,:] = np.squeeze(\
    Dataset('Port/y2006_RCP45/b2006.wcm.port.rcp45.baseline.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FSNT'])
    
    RCP45_forc_FLNR_2060[i,:,:] = np.squeeze(\
    Dataset('Port/y2060_RCP45/b2060.wcm.port.rcp45.pert.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FLNR'])
    
    RCP45_forc_FSNR_2060[i,:,:] = np.squeeze(\
    Dataset('Port/y2060_RCP45/b2060.wcm.port.rcp45.pert.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FSNR'])
    
    RCP45_forc_FLNR_2006[i,:,:] = np.squeeze(\
    Dataset('Port/y2006_RCP45/b2006.wcm.port.rcp45.baseline.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FLNR'])
    
    RCP45_forc_FSNR_2006[i,:,:] = np.squeeze(\
    Dataset('Port/y2006_RCP45/b2006.wcm.port.rcp45.baseline.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FSNR'])
    
RCP45_forc_FLNT_2060 = np.ma.masked_invalid(RCP45_forc_FLNT_2060)
RCP45_forc_FSNT_2060 = np.ma.masked_invalid(RCP45_forc_FSNT_2060)
RCP45_forc_FLNT_2006 = np.ma.masked_invalid(RCP45_forc_FLNT_2006)
RCP45_forc_FSNT_2006 = np.ma.masked_invalid(RCP45_forc_FSNT_2006)

RCP45_forc_FLNR_2060 = np.ma.masked_invalid(RCP45_forc_FLNR_2060)
RCP45_forc_FSNR_2060 = np.ma.masked_invalid(RCP45_forc_FSNR_2060)
RCP45_forc_FLNR_2006 = np.ma.masked_invalid(RCP45_forc_FLNR_2006)
RCP45_forc_FSNR_2006 = np.ma.masked_invalid(RCP45_forc_FSNR_2006)

In [32]:
RCP45_forcing_LW_TROP = -(RCP45_forc_FLNR_2060-RCP45_forc_FLNR_2006)
RCP45_forcing_SW_TROP = RCP45_forc_FSNR_2060-RCP45_forc_FSNR_2006
RCP45_forcing = RCP45_forcing_LW_TROP+RCP45_forcing_SW_TROP

RCP45_forcing_LW_TOA = -(RCP45_forc_FLNT_2060-RCP45_forc_FLNT_2006)
RCP45_forcing_SW_TOA = RCP45_forc_FSNT_2060-RCP45_forc_FSNT_2006
RCP45_forcing_NET_TOA = RCP45_forcing_LW_TOA+RCP45_forcing_SW_TOA

In [33]:
WAVD_forc_FLNT_2060 = np.zeros([12,96,144])
WAVD_forc_FSNT_2060 = np.zeros([12,96,144])
WAVD_forc_FLNR_2060 = np.zeros([12,96,144])
WAVD_forc_FSNR_2060 = np.zeros([12,96,144])

WAVD_forc_FLNT_2006 = np.zeros([12,96,144])
WAVD_forc_FSNT_2006 = np.zeros([12,96,144])
WAVD_forc_FLNR_2006 = np.zeros([12,96,144])
WAVD_forc_FSNR_2006 = np.zeros([12,96,144])
Months = ['01','02','03','04','05','06','07','08','09','10','11','12']
for i in range(12):
    WAVD_forc_FLNT_2060[i,:,:] = np.squeeze(\
    Dataset('Port/y2060_RCP45_WAVD/b2060.wcm.port.rcp45.WAVD.pert.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FLNT'])
    
    WAVD_forc_FSNT_2060[i,:,:] = np.squeeze(\
    Dataset('Port/y2060_RCP45_WAVD/b2060.wcm.port.rcp45.WAVD.pert.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FSNT'])
    
    WAVD_forc_FLNT_2006[i,:,:] = np.squeeze(\
    Dataset('Port/y2006_RCP45_WAVD/b2006.wcm.port.rcp45.WAVD.baseline.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FLNT'])
    
    WAVD_forc_FSNT_2006[i,:,:] = np.squeeze(\
    Dataset('Port/y2006_RCP45_WAVD/b2006.wcm.port.rcp45.WAVD.baseline.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FSNT'])
    
    
    WAVD_forc_FLNR_2060[i,:,:] = np.squeeze(\
    Dataset('Port/y2060_RCP45_WAVD/b2060.wcm.port.rcp45.WAVD.pert.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FLNR'])
    
    WAVD_forc_FSNR_2060[i,:,:] = np.squeeze(\
    Dataset('Port/y2060_RCP45_WAVD/b2060.wcm.port.rcp45.WAVD.pert.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FSNR'])
    
    WAVD_forc_FLNR_2006[i,:,:] = np.squeeze(\
    Dataset('Port/y2006_RCP45_WAVD/b2006.wcm.port.rcp45.WAVD.baseline.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FLNR'])
    
    WAVD_forc_FSNR_2006[i,:,:] = np.squeeze(\
    Dataset('Port/y2006_RCP45_WAVD/b2006.wcm.port.rcp45.WAVD.baseline.001.'\
            +Months[i]+'.RAD.tave.nc').variables['FSNR'])
    
WAVD_forc_FLNT_2060 = np.ma.masked_invalid(WAVD_forc_FLNT_2060)
WAVD_forc_FSNT_2060 = np.ma.masked_invalid(WAVD_forc_FSNT_2060)
WAVD_forc_FLNT_2006 = np.ma.masked_invalid(WAVD_forc_FLNT_2006)
WAVD_forc_FSNT_2006 = np.ma.masked_invalid(WAVD_forc_FSNT_2006)

WAVD_forc_FLNR_2060 = np.ma.masked_invalid(WAVD_forc_FLNR_2060)
WAVD_forc_FSNR_2060 = np.ma.masked_invalid(WAVD_forc_FSNR_2060)
WAVD_forc_FLNR_2006 = np.ma.masked_invalid(WAVD_forc_FLNR_2006)
WAVD_forc_FSNR_2006 = np.ma.masked_invalid(WAVD_forc_FSNR_2006)

In [34]:
WAVD_forcing_LW_TROP = -(WAVD_forc_FLNR_2060-WAVD_forc_FLNR_2006)
WAVD_forcing_SW_TROP = WAVD_forc_FSNR_2060-WAVD_forc_FSNR_2006
WAVD_forcing = WAVD_forcing_LW_TROP+WAVD_forcing_SW_TROP

WAVD_forcing_LW_TOA = -(WAVD_forc_FLNT_2060-WAVD_forc_FLNT_2006)
WAVD_forcing_SW_TOA = WAVD_forc_FSNT_2060-WAVD_forc_FSNT_2006
WAVD_forcing_NET_TOA = WAVD_forcing_LW_TOA+WAVD_forcing_SW_TOA

In [35]:
#read in surface temperature changes

RCP45_dST = np.mean(pickle.load(open('Future Projection CC Responses/RCP45_dST.pickle',\
                'rb'),encoding='latin1'),axis=0)
WAVD_dST = np.mean(pickle.load(open('Future Projection CC Responses/wAVD_dST.pickle',\
                'rb'),encoding='latin1'),axis=0)

In [38]:
WAVD_feedback = WAVD_forcing/WAVD_dST
RCP45_feedback = RCP45_forcing/RCP45_dST

In [46]:
WAVD_forcing_AA = AA.LatLonavg_Time(WAVD_forcing[:,80:,:],Lat[80:],Lon)
WAVD_forcing_GA = AA.LatLonavg_Time(WAVD_forcing[:,:,:],Lat[:],Lon)

RCP45_forcing_AA = AA.LatLonavg_Time(RCP45_forcing[:,80:,:],Lat[80:],Lon)
RCP45_forcing_GA = AA.LatLonavg_Time(RCP45_forcing[:,:,:],Lat[:],Lon)

WAVD_feedback_AA = AA.LatLonavg_Time(WAVD_feedback[:,80:,:],Lat[80:],Lon)
WAVD_feedback_GA = AA.LatLonavg_Time(WAVD_feedback[:,:,:],Lat[:],Lon)

RCP45_feedback_AA = AA.LatLonavg_Time(RCP45_feedback[:,80:,:],Lat[80:],Lon)
RCP45_feedback_GA = AA.LatLonavg_Time(RCP45_feedback[:,:,:],Lat[:],Lon)

In [67]:
WAVD_forcing_AA_Annual = np.mean(WAVD_forcing_AA)
WAVD_forcing_AA_MAM = np.mean(WAVD_forcing_AA[2:5])
WAVD_forcing_AA_JJA = np.mean(WAVD_forcing_AA[5:8])
WAVD_forcing_AA_SON = np.mean(WAVD_forcing_AA[8:11])
WAVD_forcing_AA_DJF = np.mean((WAVD_forcing_AA[0],WAVD_forcing_AA[1],WAVD_forcing_AA[11]))

WAVD_feedback_AA_Annual = np.mean(WAVD_feedback_AA)
WAVD_feedback_AA_MAM = np.mean(WAVD_feedback_AA[2:5])
WAVD_feedback_AA_JJA = np.mean(WAVD_feedback_AA[5:8])
WAVD_feedback_AA_SON = np.mean(WAVD_feedback_AA[8:11])
WAVD_feedback_AA_DJF = np.mean((WAVD_feedback_AA[0],WAVD_feedback_AA[1],WAVD_feedback_AA[11]))

In [68]:
RCP45_forcing_AA_Annual = np.mean(RCP45_forcing_AA)
RCP45_forcing_AA_MAM = np.mean(RCP45_forcing_AA[2:5])
RCP45_forcing_AA_JJA = np.mean(RCP45_forcing_AA[5:8])
RCP45_forcing_AA_SON = np.mean(RCP45_forcing_AA[8:11])
RCP45_forcing_AA_DJF = np.mean((RCP45_forcing_AA[0],RCP45_forcing_AA[1],RCP45_forcing_AA[11]))

RCP45_feedback_AA_Annual = np.mean(RCP45_feedback_AA)
RCP45_feedback_AA_MAM = np.mean(RCP45_feedback_AA[2:5])
RCP45_feedback_AA_JJA = np.mean(RCP45_feedback_AA[5:8])
RCP45_feedback_AA_SON = np.mean(RCP45_feedback_AA[8:11])
RCP45_feedback_AA_DJF = np.mean((RCP45_feedback_AA[0],RCP45_feedback_AA[1],RCP45_feedback_AA[11]))

In [70]:
wAVD_RF_EnergyB_AA_file = open('wAVD_RF_EnergyB_AA.pickle','wb')
pickle.dump(WAVD_forcing_AA_Annual,wAVD_RF_EnergyB_AA_file)
wAVD_RF_EnergyB_AA_file.close()
wAVD_RF_EnergyB_MAM_AA_file = open('wAVD_RF_EnergyB_MAM_AA.pickle','wb')
pickle.dump(WAVD_forcing_AA_MAM,wAVD_RF_EnergyB_MAM_AA_file)
wAVD_RF_EnergyB_MAM_AA_file.close()
wAVD_RF_EnergyB_JJA_AA_file = open('wAVD_RF_EnergyB_JJA_AA.pickle','wb')
pickle.dump(WAVD_forcing_AA_JJA,wAVD_RF_EnergyB_JJA_AA_file)
wAVD_RF_EnergyB_JJA_AA_file.close()
wAVD_RF_EnergyB_SON_AA_file = open('wAVD_RF_EnergyB_SON_AA.pickle','wb')
pickle.dump(WAVD_forcing_AA_SON,wAVD_RF_EnergyB_SON_AA_file)
wAVD_RF_EnergyB_SON_AA_file.close()
wAVD_RF_EnergyB_DJF_AA_file = open('wAVD_RF_EnergyB_DJF_AA.pickle','wb')
pickle.dump(WAVD_forcing_AA_DJF,wAVD_RF_EnergyB_DJF_AA_file)
wAVD_RF_EnergyB_DJF_AA_file.close()

wAVD_RF_Feedback_AA_file = open('wAVD_RF_Feedback_AA.pickle','wb')
pickle.dump(WAVD_feedback_AA_Annual,wAVD_RF_Feedback_AA_file)
wAVD_RF_Feedback_AA_file.close()
wAVD_RF_Feedback_MAM_AA_file = open('wAVD_RF_Feedback_MAM_AA.pickle','wb')
pickle.dump(WAVD_feedback_AA_MAM,wAVD_RF_Feedback_MAM_AA_file)
wAVD_RF_Feedback_MAM_AA_file.close()
wAVD_RF_Feedback_JJA_AA_file = open('wAVD_RF_Feedback_JJA_AA.pickle','wb')
pickle.dump(WAVD_feedback_AA_JJA,wAVD_RF_Feedback_JJA_AA_file)
wAVD_RF_Feedback_JJA_AA_file.close()
wAVD_RF_Feedback_SON_AA_file = open('wAVD_RF_Feedback_SON_AA.pickle','wb')
pickle.dump(WAVD_feedback_AA_SON,wAVD_RF_Feedback_SON_AA_file)
wAVD_RF_Feedback_SON_AA_file.close()
wAVD_RF_Feedback_DJF_AA_file = open('wAVD_RF_Feedback_DJF_AA.pickle','wb')
pickle.dump(WAVD_feedback_AA_DJF,wAVD_RF_Feedback_DJF_AA_file)
wAVD_RF_Feedback_DJF_AA_file.close()

In [71]:
RCP45_RF_EnergyB_AA_file = open('RCP45_RF_EnergyB_AA.pickle','wb')
pickle.dump(RCP45_forcing_AA_Annual,RCP45_RF_EnergyB_AA_file)
RCP45_RF_EnergyB_AA_file.close()
RCP45_RF_EnergyB_MAM_AA_file = open('RCP45_RF_EnergyB_MAM_AA.pickle','wb')
pickle.dump(RCP45_forcing_AA_MAM,RCP45_RF_EnergyB_MAM_AA_file)
RCP45_RF_EnergyB_MAM_AA_file.close()
RCP45_RF_EnergyB_JJA_AA_file = open('RCP45_RF_EnergyB_JJA_AA.pickle','wb')
pickle.dump(RCP45_forcing_AA_JJA,RCP45_RF_EnergyB_JJA_AA_file)
RCP45_RF_EnergyB_JJA_AA_file.close()
RCP45_RF_EnergyB_SON_AA_file = open('RCP45_RF_EnergyB_SON_AA.pickle','wb')
pickle.dump(RCP45_forcing_AA_SON,RCP45_RF_EnergyB_SON_AA_file)
RCP45_RF_EnergyB_SON_AA_file.close()
RCP45_RF_EnergyB_DJF_AA_file = open('RCP45_RF_EnergyB_DJF_AA.pickle','wb')
pickle.dump(RCP45_forcing_AA_DJF,RCP45_RF_EnergyB_DJF_AA_file)
RCP45_RF_EnergyB_DJF_AA_file.close()

RCP45_RF_Feedback_AA_file = open('RCP45_RF_Feedback_AA.pickle','wb')
pickle.dump(RCP45_feedback_AA_Annual,RCP45_RF_Feedback_AA_file)
RCP45_RF_Feedback_AA_file.close()
RCP45_RF_Feedback_MAM_AA_file = open('RCP45_RF_Feedback_MAM_AA.pickle','wb')
pickle.dump(RCP45_feedback_AA_MAM,RCP45_RF_Feedback_MAM_AA_file)
RCP45_RF_Feedback_MAM_AA_file.close()
RCP45_RF_Feedback_JJA_AA_file = open('RCP45_RF_Feedback_JJA_AA.pickle','wb')
pickle.dump(RCP45_feedback_AA_JJA,RCP45_RF_Feedback_JJA_AA_file)
RCP45_RF_Feedback_JJA_AA_file.close()
RCP45_RF_Feedback_SON_AA_file = open('RCP45_RF_Feedback_SON_AA.pickle','wb')
pickle.dump(RCP45_feedback_AA_SON,RCP45_RF_Feedback_SON_AA_file)
RCP45_RF_Feedback_SON_AA_file.close()
RCP45_RF_Feedback_DJF_AA_file = open('RCP45_RF_Feedback_DJF_AA.pickle','wb')
pickle.dump(RCP45_feedback_AA_DJF,RCP45_RF_Feedback_DJF_AA_file)
RCP45_RF_Feedback_DJF_AA_file.close()